<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Extracting-tweets-from-SQL-database" data-toc-modified-id="Extracting-tweets-from-SQL-database-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting tweets from SQL database</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Word2vec" data-toc-modified-id="Word2vec-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Word2vec</a></span></li></ul></li></ul></li><li><span><a href="#Topic-Modeling" data-toc-modified-id="Topic-Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Topic Modeling</a></span></li></ul></div>

# Libraries 

    The following Libraries were used in the development of this project:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
import nltk
import pyodbc
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
import json 

import warnings
warnings.filterwarnings("ignore")


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# Data Preprocessing

The data preprocessing consist of different stages:

In [22]:
import nltk 


'''Natural Language Processing libraries'''
import nltk 
import gensim
import regex as re
import spacy
from spacy.lang.en import English
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader as api
import re, string, unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from gensim.models import FastText
from gensim.models import Word2Vec



class Document:
    """ Retrieve the narratives from the DataFrame and respectively
        store and pre-process it. 
        
        :param df: DataFrame including the reports and the predictor variable. 
        
        
        :ivar data: Stores the DataFrame.
        :ivar text: Stores the narratives as string.
        :ivar corpus: Stores the pre-processed text.
    """
    
    
    def __init__(self, df):
        self.data = df
        self.text = df["text"].astype(str)
        self.textPreProcessing()
        
        
    def remove_non_ascii(self, words):
        """Remove non-ASCII characters from list of tokenized words
        
        :param words:  List of words to be transformed when removing non_ascii characters.
        
        :return new_words: List of words after the transformation of removed non_ascii characters.
        
        """
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words


    def remove_punctuation(self, words):
        """Remove punctuation from list of tokenized words
        
        :param words:  List of words that will get remove their punctuations, if any. 
        
        :return new_words: List of transformed words.
        
        
        """
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words


    def stem_words(self, words):
        """Stem words in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective stems.
        
        
        """
        
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        return stems
    
    def lemmatize_verbs(self, words):
        """Lemmatize verbs in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective lemmas.
        
        """
        
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return lemmas
    
    
    
    def remove_stopwords(self, words):
        """Remove common words that have no meaning or importance in the sentence.

        :param words:  List of words to be processed and get stop words removed.. 

        :return new_words: List of words with the stop words already removed."""
            
        
        stop_words = set(stopwords.words('spanish')) 
        stop_words1 = set(stopwords.words('english')) 


        
        for word in stop_words:
            if word in words:
                words.remove(word)
                
        for word in stop_words1:
            if word in words:
                words.remove(word)
                
        return words


    
    def normalize(self, words):
        words = self.remove_non_ascii(words)
        words = self.remove_stopwords(words)
        words = self.remove_punctuation(words)
        words = self.lemmatize_verbs(words)
        return words
    
    
    def textPreProcessing(self):
        """Pre-process the text, normalize and clean it.
        The function stores the cleaned text in the self.data
        attribute. """
        p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)


        clean_text = []

        for narrative in self.text:
            sentence = p.clean(narrative)
            #sentence = re.sub('RT @[\w_]+:', '', sentence)

            #sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
            sentence = word_tokenize(sentence)
            sentence = self.normalize(sentence)
                
                
            clean_text.append(sentence)
            
            
                    
        print(len(self.text), len(clean_text))
        self.data["clean_text"] = clean_text

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def concatTweets(table):
    tweets = ''
    c = 0
    for row in df.tweets:
        if(c == 0):
            c+= 1
            tweets = pd.DataFrame(json.loads(row))
        else:
            tweets = pd.concat([tweets, pd.DataFrame(json.loads(row))], axis=0, ignore_index=True)
    
    
    return tweets 

# Extracting tweets from SQL database

In [5]:
server = 'sqldatamining.database.windows.net'
database = 'SNA'
username = 'UserAdmin'
password = 'Machomen123'   
driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sqldatamining.database.windows.net,1433', user='UserAdmin' , password='Machomen123', database='SNA')




In [6]:
cursor = cnxn.cursor()

In [118]:
rows = cursor.execute("SELECT * from dbo.Users1 ").fetchmany(40)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [119]:
df = pd.DataFrame((tuple(t) for t in rows), columns=['id', 
 'author',
 'created_at',
 'location',
 'description',
 'verified',
 'followers',
 'following',
 'favourites_count',
 'statuses_count',
 'lang',
 'tweets',
 'following_json',
 'followers_json']) 


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [99]:
df.head()

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
0,15224855,ricardo_l,2008-06-24 22:36:49,"San Juan, Puerto Rico","Policy and Communications, Stakeholder Managem...",False,922,951,3670,3095,None,"[{""id"": 1355584663377612802, ""created_at"": ""20...","[{""id"": 1197230058039054339, ""author"": ""andgon...","[{""id"": 107615345, ""author"": ""dramosgutierrez""..."
1,15851028,mateo922,2008-08-14 14:48:08,"Cayey, Puerto Rico",I say what I do and I do what I said.,False,79,526,9135,2023,None,"[{""id"": 1355177247951478785, ""created_at"": ""20...","[{""id"": 2203150838, ""author"": ""TeanaKitten"", ""...","[{""id"": 1381820379891695618, ""author"": ""ousman..."
2,17977795,Badillo83,2008-12-08 23:55:49,"Aguadilla, PR",,False,44,125,29,8,None,"[{""id"": 1340802217306144768, ""created_at"": ""20...","[{""id"": 1357812393729003520, ""author"": ""Crypto...","[{""id"": 75538366, ""author"": ""AdrianAcostaPR"", ..."
3,17977795,Badillo83,2008-12-08 23:55:49,"Aguadilla, PR",,False,44,125,29,8,None,"[{""id"": 1340802217306144768, ""created_at"": ""20...","[{""id"": 1357812393729003520, ""author"": ""Crypto...","[{""id"": 75538366, ""author"": ""AdrianAcostaPR"", ..."
4,17977795,Badillo83,2008-12-08 23:55:49,"Aguadilla, PR",,False,44,125,30,8,None,"[{""id"": 1340802217306144768, ""created_at"": ""20...","[{""id"": 1357812393729003520, ""author"": ""Crypto...","[{""id"": 75538366, ""author"": ""AdrianAcostaPR"", ..."


In [120]:
index = 0
for tweets in df.tweets:
    print(len(tweets))
    if(len(tweets) == 2):
        df.drop([index], inplace=True, axis=0)
    
    index+=1
    
df.reset_index(inplace=True)

index = 0
temp_user = ""
for user in df.author:
    if(user != temp_user):
        print("a")
        temp_user = user 
    else:
        df.drop([index], inplace=True, axis=0)
    
    
    index+=1
    
    
    

435940
181519
2
34422
2
34422
2
2
68260
331733
2
2
826569
2
476
617388
60177
2
725156
763868
731587
494057
2
2
12645
498511
398221
2
542318
325
325
325
702299
685193
2
2
2
140497
261657
685293
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
df.head(10)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
0,0,2284857094,themoonisironic,2014-01-16 01:10:33,‘97 // they/them // ????????,"where there is hope, there are trials // art: ...",False,101,516,121166,36377,None,"[{""id"": 1356030533780824069, ""created_at"": ""20...","[{""id"": 1387079251476815875, ""author"": ""rikuno...","[{""id"": 1384297303129825284, ""author"": ""bts123..."
1,1,834324620,bbelita23,2012-09-19 23:21:48,"spooky town, PR",just put it out into the universe,False,670,443,33428,92917,None,"[{""id"": 1356026926134140928, ""created_at"": ""20...","[{""id"": 2513536418, ""author"": ""Kvn_Astacio"", ""...","[{""id"": 1289669319526420482, ""author"": ""keviin..."
2,3,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
4,8,1346688657365889024,SJCiudadCapital,2021-01-06 05:23:04,,Cuenta Oficial de la Ciudad Capital de Puerto ...,False,2915,183,291,1157,None,"[{""id"": 1355954464436473856, ""created_at"": ""20...","[{""id"": 1309256724830969857, ""author"": ""estefa...","[{""id"": 1271152649304518657, ""author"": ""JCruz_..."
5,9,521836439,Angelicv_G,2012-03-12 01:12:16,Puerto Rico,UT • yo no sé que poner aquí??,False,962,519,115665,130883,None,"[{""id"": 1356041673227313154, ""created_at"": ""20...","[{""id"": 1192074658994180096, ""author"": ""Nsnili...","[{""id"": 1383464766283280389, ""author"": ""geegee..."


In [121]:
x = concatTweets(df)




/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import preprocessor as p
p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)
p.clean('Preprocessor í is #awesome 👍 https://github.com/s/preprocessor')

'Preprocessor í is 👍'

In [56]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')


texto = pd.DataFrame(tuits, columns=["text"])

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\afabi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\afabi/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\afabi/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [57]:
nltk.download('stopwords')
nltk.download('wordnet')
documento = Document(texto)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\afabi/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\afabi/nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


AttributeError: module 'preprocessor' has no attribute 'set_options'

In [502]:
documento.data.head()

,text,clean_text
0,Nada más cierto! 🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️,"[Nada, mas, cierto]"
1,Llamen a mi jefe diganle que la lluvia no me d...,"[Llamen, jefe, diganle, lluvia, deja, ir, trab..."
2,:,[]
3,": , , Pescao! Pescao!","[Pescao, Pescao]"
4,Así está bien o todavía no estas tan sorprendi...,"[Asi, bien, todavia, tan, sorprendido, El, gol..."


### Word2vec

In [356]:
model = Word2Vec(documento.data["clean_text"], min_count=0, workers=20, window=2,  alpha=0.02, hs=1)



In [503]:
my_dict = dict({})
for idx, key in enumerate(model.wv.key_to_index):
    my_dict[key] = model.wv[key]

In [504]:
vectores = []
for a in documento.data.clean_text:
    t  = []
    for word in a:
        try:
            t.append(my_dict[word])
        except:
            continue
    
    vectores.append(t)
        
documento.data["vectorized_text"] = vectores
documento.data.tail()

,text,clean_text,vectorized_text
113,": El ajusta la magnitud a , en el límite de pr...","[El, ajusta, magnitud, limite, producir, tsunami]","[[-0.017272485, 0.008652826, 0.007515971, 0.00..."
114,: “ Van a seguir ocurriendo eventos fuertes. H...,"[Van, seguir, ocurriendo, eventos, fuertes, Ha...","[[-0.0027897793, -0.0016228552, -0.007687815, ..."
115,: Casa colapsada en Yauco.,"[Casa, colapsada, Yauco]","[[0.008535036, 0.013252284, 0.00035941493, 0.0..."
116,: “Las réplicas podrían ser de magnitud mayor ...,"[Las, replicas, podrian, ser, magnitud, mayor,...","[[0.0029013832, -0.0027108788, -0.0015446608, ..."
117,": PRELIMINAR -01-06 :58:03 No hay Aviso, Adver...","[PRELIMINAR, 06, 5803, No, Aviso, Advertencia,...","[[-0.0127012255, 0.0040803654, 0.009027934, -0..."


In [505]:
df.tail()

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
5,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
6,1388202983964520449,Personn34091581,2021-04-30 18:46:05,Costa brava,,False,0,2,9,7,None,[],"[{""id"": 560803492, ""author"": ""UrbanLePharaon"",...",[]
7,908687401123553280,rsantanafonseca,2017-09-15 13:42:13,"COPU, UPRRP","19 (+3) | ?????? | Pop culture enthusiast, soc...",False,891,620,121676,124488,None,[],"[{""id"": 755882683, ""author"": ""_vidalysrms"", ""c...","[{""id"": 1344047685716815872, ""author"": ""LeQuee..."
8,1346688657365889024,SJCiudadCapital,2021-01-06 05:23:04,,Cuenta Oficial de la Ciudad Capital de Puerto ...,False,2915,183,291,1157,None,"[{""id"": 1355954464436473856, ""created_at"": ""20...","[{""id"": 1309256724830969857, ""author"": ""estefa...","[{""id"": 1271152649304518657, ""author"": ""JCruz_..."
9,521836439,Angelicv_G,2012-03-12 01:12:16,Puerto Rico,UT • yo no sé que poner aquí??,False,962,519,115665,130883,None,"[{""id"": 1356041673227313154, ""created_at"": ""20...","[{""id"": 1192074658994180096, ""author"": ""Nsnili...","[{""id"": 1383464766283280389, ""author"": ""geegee..."


# Topic Modeling 

In [158]:
def asignTopics(users, tweetss):
    topics = userSubSets(tweetss)
    topic0 = []
    topic1 = []
    topic2 = []
    topic3 = []
    topic4 = []
    
    for topic in topics:
        topic0.append(topic[0])
        topic1.append(topic[1])
        topic2.append(topic[2])
        topic3.append(topic[3])
        topic4.append(topic[4])
        
        
    users["Topic0"] = np.asarray(topic0)
    users["Topic1"] = np.asarray(topic1)
    users["Topic2"] = np.asarray(topic2)
    users["Topic3"] = np.asarray(topic3)
    users["Topic4"] = np.asarray(topic4)
    
    
    return users






def userSubSets(tweets_df):
    user = ''
    topics = []
    for instance in tweets_df.user:
        if(user != instance):
            print(instance)
            user = instance
            actual_user =  tweets_df['user']== user
            user_subset = tweets_df[actual_user]
            topics.append(topicModeling(user_subset))
        else:
            continue
            
            
    return topics
            

            
            
def topicModeling(user_subset):
    tweets = Document(pd.DataFrame(user_subset))
    id2word = corpora.Dictionary(tweets.data.clean_text)

    # Create Corpus
    texts = tweets.data.clean_text

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    a = lda_model.get_topics()
    vector = np.vectorize(np.float)
    x = vector(a)
    
    print(lda_model.print_topics())
    
    
    return a

    
    


    
    
            
        
        
    

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [159]:
users = asignTopics(df, x)



/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


themoonisironic
1538 1538


<ipython-input-158-c79bb0d60958>:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.vectorize(np.float)


[(0, '0.035*"u" + 0.023*"like" + 0.022*"stream" + 0.020*"srr" + 0.017*"s" + 0.016*"i" + 0.013*"love" + 0.013*"today" + 0.012*"play" + 0.010*"hear"'), (1, '0.025*"like" + 0.021*"fkn" + 0.020*"make" + 0.019*"i" + 0.016*"soraru" + 0.014*"be" + 0.014*"even" + 0.014*"look" + 0.012*"sound" + 0.011*"s"'), (2, '0.021*"S" + 0.017*"IS" + 0.017*"HE" + 0.017*"I" + 0.015*"good" + 0.015*"A" + 0.014*"THE" + 0.013*"WHAT" + 0.013*"IT" + 0.013*"T"'), (3, '0.020*"the" + 0.018*"to" + 0.016*"one" + 0.015*"say" + 0.015*"know" + 0.012*"sing" + 0.012*"like" + 0.011*"i" + 0.010*"show" + 0.009*"im"'), (4, '0.029*"i" + 0.021*"get" + 0.021*"want" + 0.016*"man" + 0.015*"go" + 0.014*"sakata" + 0.013*"watch" + 0.012*"chainsaw" + 0.010*"shxt" + 0.010*"every"')]
bbelita23
719 719


<ipython-input-158-c79bb0d60958>:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.vectorize(np.float)


...

<ipython-input-158-c79bb0d60958>:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.vectorize(np.float)


[(0, '0.029*"I" + 0.024*"God" + 0.012*"se" + 0.012*"Es" + 0.011*"thank" + 0.011*"Lo" + 0.010*"bless" + 0.009*"Just" + 0.009*"finish" + 0.009*"every"'), (1, '0.019*"the" + 0.016*"breath" + 0.012*"El" + 0.012*"Gracias" + 0.011*"Dios" + 0.011*"en" + 0.011*"do" + 0.011*"tiempo" + 0.010*"Pero" + 0.010*"that"'), (2, '0.020*"you" + 0.020*"be" + 0.011*"New" + 0.011*"people" + 0.010*"vez" + 0.010*"time" + 0.009*"You" + 0.009*"alla" + 0.008*"navidad" + 0.008*"Cuando"'), (3, '0.028*"de" + 0.012*"puede" + 0.011*"Eso" + 0.011*"online" + 0.011*"Este" + 0.010*"seguir" + 0.010*"dando" + 0.009*"proyecto" + 0.009*"clave" + 0.009*"conocer"'), (4, '0.017*"dias" + 0.013*"bajo" + 0.012*"en" + 0.012*"Bayamon" + 0.010*"varios" + 0.010*"pr" + 0.010*"llevamos" + 0.010*"hogares" + 0.010*"Saben" + 0.010*"voltaje"')]
nnavm_og
1022 1022


<ipython-input-158-c79bb0d60958>:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.vectorize(np.float)


[(0, '0.022*"Que" + 0.018*"vez" + 0.018*"si" + 0.018*"quiero" + 0.014*"persona" + 0.012*"problema" + 0.012*"Tal" + 0.012*"platos" + 0.011*"que" + 0.010*"Y"'), (1, '0.029*"ver" + 0.015*"DE" + 0.014*"A" + 0.013*"nena" + 0.012*"mas" + 0.010*"ES" + 0.010*"LOS" + 0.009*"Si" + 0.009*"gt" + 0.009*"hace"'), (2, '0.024*"personas" + 0.022*"que" + 0.022*"ano" + 0.020*"No" + 0.019*"mismo" + 0.017*"fin" + 0.015*"solo" + 0.015*"dia" + 0.013*"La" + 0.012*"al"'), (3, '0.024*"te" + 0.019*"la" + 0.015*"Ya" + 0.014*"Me" + 0.013*"tan" + 0.013*"feliz" + 0.013*"si" + 0.011*"hoy" + 0.011*"Estoy" + 0.011*"puedo"'), (4, '0.036*"voy" + 0.024*"si" + 0.023*"unico" + 0.017*"Yo" + 0.012*"de" + 0.012*"pa" + 0.010*"alguien" + 0.010*"ahora" + 0.009*"pedir" + 0.008*"envio"')]
23jamel
2747 2747
[(0, '0.027*"DE" + 0.024*"Y" + 0.017*"A" + 0.016*"LA" + 0.014*"QUE" + 0.013*"Lo" + 0.012*"EL" + 0.011*"merengue" + 0.008*"solo" + 0.008*"sabe"'), (1, '0.027*"Me" + 0.027*"La" + 0.015*"gente" + 0.013*"Yo" + 0.013*"Los" + 0.011*"ca

<ipython-input-158-c79bb0d60958>:71: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vector = np.vectorize(np.float)


In [151]:
topics = df[["Topic0", "Topic1", "Topic2", "Topic3", "Topic4"]]
topics.head()



/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic0,Topic1,Topic2,Topic3,Topic4
0,"[6.4059626e-05, 6.423177e-05, 6.522911e-05, 6....","[7.131866e-05, 7.13527e-05, 7.226892e-05, 7.27...","[6.306197e-05, 0.016837869, 6.1405466e-05, 6.1...","[0.00010639355, 7.018809e-05, 0.0017958591, 0....","[6.8705805e-05, 6.8814385e-05, 7.199822e-05, 6..."
1,"[0.00022911737, 0.00022911589, 0.00022911589, ...","[0.000234279, 0.00023427073, 0.00023427073, 0....","[0.00019619269, 0.00019618758, 0.00019618758, ...","[0.00049253425, 0.00020517374, 0.00020517374, ...","[0.00019165115, 0.00046023697, 0.00046023697, ..."
2,"[0.0007776633, 0.0007776634, 0.004510385, 0.00...","[0.00044708577, 0.00044708565, 0.00044708583, ...","[0.0008799134, 0.0008799134, 0.0051034573, 0.0...","[0.0005133346, 0.0005133345, 0.0005133347, 0.0...","[0.003119646, 0.005321997, 0.0031196459, 0.000..."
4,"[0.00019731079, 0.00639439, 0.0001994974, 0.04...","[0.009712919, 0.0061110817, 0.009712921, 0.020...","[0.0002943054, 0.0002943031, 0.0002943054, 0.0...","[0.00035719664, 0.00035717327, 0.00035719745, ...","[0.0004296852, 0.00042968814, 0.0014735191, 0...."
5,"[8.2627594e-05, 8.2628765e-05, 8.2628765e-05, ...","[8.410271e-05, 8.4103274e-05, 8.4103274e-05, 8...","[7.250998e-05, 7.2512456e-05, 7.2512456e-05, 7...","[6.990478e-05, 6.990746e-05, 6.990746e-05, 8.2...","[0.0003292375, 0.0001769559, 0.0001769559, 0.0..."


In [25]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
lda_model.get_topics()[5]

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


IndexError: index 5 is out of bounds for axis 0 with size 5

In [28]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.126004  0.094526       1        1  21.899446
4     -0.198367  0.066284       2        1  21.896521
3     -0.001912 -0.185583       3        1  19.960156
1      0.038494 -0.001149       4        1  19.031201
0      0.035780  0.025922       5        1  17.212676, topic_info=       Term       Freq      Total Category  logprob  loglift
5         I  34.000000  34.000000  Default  30.0000  30.0000
42     love  22.000000  22.000000  Default  29.0000  29.0000
137     que  20.000000  20.000000  Default  28.0000  28.0000
1726  claim  14.000000  14.000000  Default  27.0000  27.0000
935     los  15.000000  15.000000  Default  26.0000  26.0000
...     ...        ...        ...      ...      ...      ...
1700  Girls   1.915179   2.462405   Topic5  -5.6305   1.5082
1701    guy   1.915179   2.462405   Topic5  -5.6305   1.5082
1702  sleep   1.915179   2.462405   Topic5  -5.6305   1.5082
1703   zone   1.915179   2.462405   Topic5  -5.6305   1.5082
803      en   2.377364   3.196787   Topic5  -5.4143   1.4634

[192 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1478      5  0.738114       
583       1  0.185664      A
583       3  0.742656      A
1725      4  0.902131    Asi
534       3  0.943928   Dios
...     ...       ...    ...
293       4  0.840526   want
428       1  0.777816  woman
1173      1  0.941177   year
294       4  0.927333    you
1703      5  0.812214   zone

[168 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 1])

In [359]:
word2vec_model.wv.key_to_index

{'de': 0,
 'No': 1,
 'a': 2,
 'Aviso': 3,
 'Bayamon': 4,
 'Nuevo': 5,
 'Barrio': 6,
 'El': 7,
 'la': 8,
 'magnitud': 9,
 'Ml': 10,
 'Sentido': 11,
 'Km': 12,
 'Prof': 13,
 'Mag': 14,
 'PRELIMINAR': 15,
 'Max': 16,
 'Puerto': 17,
 'Rico': 18,
 'Int': 19,
 'temblor': 20,
 'PR': 21,
 'enero2020': 22,
 'PM': 23,
 'que': 24,
 'mas': 25,
 'tsunami': 26,
 'be': 27,
 'sabado': 28,
 'luz': 29,
 'servicio': 30,
 'hoy': 31,
 'La': 32,
 'Gracias': 33,
 'los': 34,
 'Wanda': 35,
 'Hoy': 36,
 'mayor': 37,
 'mejor': 38,
 'ser': 39,
 'el': 40,
 'vez': 41,
 'Que': 42,
 'Mayaguez': 43,
 'gente': 44,
 'fuerte': 45,
 'ver': 46,
 'si': 47,
 'l': 48,
 'Mas': 49,
 'cada': 50,
 'ir': 51,
 'chicharra': 52,
 'con': 53,
 'preliminar': 54,
 'Islas': 55,
 'Advertencia': 56,
 'Vigilancia': 57,
 'no': 58,
 'Los': 59,
 'ACTUALIZADA': 60,
 'temblores': 61,
 'pandemia': 62,
 'Lo': 63,
 'Enero': 64,
 'tan': 65,
 'espero': 66,
 'isla': 67,
 'Hola': 68,
 'cierto': 69,
 'Pescao': 70,
 'bien': 71,
 'vi': 72,
 'NO': 73,
 'ent